In [1]:
spark

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 6, Finished, Available)

In [15]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 20, Finished, Available)

## Import Libraries

In [ ]:
import os
from pyspark.sql.functions import col, count, when
import pandas as pd
from pyspark.sql.functions import length
from pyspark.sql import functions as F

## Read Data

In [16]:
datastore = 'azureml://datastores/workspaceblobstore/paths/'

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 21, Finished, Available)

In [17]:
submissions_path = 'filtered-submissions'
submissions_df = spark.read.parquet(f"{datastore}{submissions_path}")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 22, Finished, Available)

## Data Processing:

In [6]:
PLOT_DIR = os.path.join("Users/sk2224/fall-2023-reddit-project-team-34/data", "plots")
CSV_DIR = os.path.join("Users/sk2224/fall-2023-reddit-project-team-34/data", "csv")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 11, Finished, Available)

Drop unnecessary columns:

In [5]:
# # List of columns to be dropped
# columns_to_drop = [
#     'promoted', 'promoted_by', 'promoted_display_name', 'promoted_url',
#     'secure_media', 'secure_media_embed', 'thumbnail', 'thumbnail_height', 
#     'thumbnail_width', 'third_party_trackers', 'third_party_tracking', 
#     'third_party_tracking_2', 'source', 'retrieved_on', 'suggested_sort', 
#     'spoiler', 'stickied', 'whitelist_status'
# ]

# # Dropping the columns
# submissions_df = submissions_df.drop(*columns_to_drop)

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 10, Finished, Available)

Shape of the data:

In [4]:
# submissions_row_count = submissions_df.count()
# submissions_col_count = len(submissions_df.columns)
# print(f"Shape of the submissions dataframe is {submissions_row_count:,}x{submissions_col_count}")

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 9, Finished, Available)

Breakdown of subreddit counts:

In [62]:
# subreddit_count_df = submissions_df.groupBy('subreddit').count().toPandas()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 67, Finished, Available)

In [6]:
submissions_df.groupBy('subreddit').count().show()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 35, 11, Finished, Available)

+--------------------+------+
|           subreddit| count|
+--------------------+------+
|               anime|404298|
|          television| 89586|
|televisionsuggest...|  7991|
|              movies|382085|
|        Animesuggest| 74101|
|    MovieSuggestions| 58907|
+--------------------+------+



In [65]:
# subreddit_count_df.to_csv(f"{CSV_DIR}/subreddit_count.csv")

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 70, Finished, Available)

Number of partitions in RDD:

In [6]:
# get number of partitions
submissions_df.rdd.getNumPartitions()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 11, Finished, Available)

100

Columns in the data:

In [7]:
# print schema
submissions_df.printSchema()

StatementMeta(87608f1f-5281-477c-8a41-f00a9ddf3071, 2, 12, Finished, Available)

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

Subset the data for faster processing:

In [6]:
submissions_df_small = submissions_df.sample(withReplacement=False, fraction=0.01)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 40, 11, Finished, Available)

In [7]:
# take a subset of columns
df = submissions_df_small.select("subreddit", "author", "title", "selftext",
                             "created_utc", "num_comments", "score", 
                             "over_18", "media", "pinned", "locked", 
                             "disable_comments", "domain", "hidden", 
                             "distinguished", "hide_score")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 40, 12, Finished, Available)

In [18]:
# take a subset of columns
df = submissions_df.select("subreddit", "author", "title", "selftext",
                             "created_utc", "num_comments", "score", 
                             "over_18", "media", "pinned", "locked", 
                             "disable_comments", "domain", "hidden", 
                             "distinguished", "hide_score")

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 23, Finished, Available)

Count the missing values in data:

In [12]:
# find missing values
missing_vals = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 17, Finished, Available)

In [13]:
missing_vals.show()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 18, Finished, Available)

+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+
|subreddit|author|title|selftext|created_utc|num_comments|score|over_18| media|pinned|locked|disable_comments|domain|hidden|distinguished|hide_score|
+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+
|        0|     0|    0|       0|          0|           0|    0|      0|892405|     0|     0|         1016968|  8952|     0|      1015530|         0|
+---------+------+-----+--------+-----------+------------+-----+-------+------+------+------+----------------+------+------+-------------+----------+



In [15]:
# Manually inputting data
data = {
    'subreddit': [0],
    'author': [0],
    'title': [0],
    'selftext': [0],
    'created_utc': [0],
    'num_comments': [0],
    'score': [0],
    'over_18': [0],
    'media': [892405],
    'pinned': [0],
    'locked': [0],
    'disable_comments': [1016968],
    'domain': [8952],
    'hidden': [0],
    'distinguished': [1015530],
    'hide_score': [0]
}

# Creating a Pandas DataFrame
missing_vals_pd = pd.DataFrame(data)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 20, Finished, Available)

In [16]:
missing_vals_pd

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 21, Finished, Available)

,subreddit,author,title,selftext,created_utc,num_comments,score,over_18,media,pinned,locked,disable_comments,domain,hidden,distinguished,hide_score
0,0,0,0,0,0,0,0,0,892405,0,0,1016968,8952,0,1015530,0


In [13]:
# missing_vals_pd = missing_vals.toPandas()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 35, 18, Finished, Available)

In [17]:
# Convert the DataFrame to long format
df_long = missing_vals_pd.melt(var_name='Column', value_name='Missing Values')

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 22, Finished, Available)

In [18]:
df_long

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 23, Finished, Available)

,Column,Missing Values
0,subreddit,0
1,author,0
2,title,0
3,selftext,0
4,created_utc,0
5,num_comments,0
6,score,0
7,over_18,0
8,media,892405
9,pinned,0


In [20]:
df_long.to_csv(f"{CSV_DIR}/num_missing_val.csv", index=False)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 36, 25, Finished, Available)

### Feature Engineering

Counting the length of the posted submission:

In [19]:
# calculate post length
df = df.withColumn('post_length', length(df.title) + length(df.selftext))

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 24, Finished, Available)

Extracting meaningful information from datetime column:

In [20]:
# convert to datetime
df = df.withColumn('created_utc', F.to_timestamp('created_utc'))

# Extract time-based features
df = df.withColumn('hour_of_day', F.hour('created_utc'))
df = df.withColumn('day_of_week', F.dayofweek('created_utc'))  # 1 (Sunday) to 7 (Saturday)
# Map each day of the week from numeric to string
df = df.withColumn('day_of_week_str', F.expr("""
    CASE day_of_week 
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END
"""))
df = df.withColumn('day_of_month', F.dayofmonth('created_utc'))
df = df.withColumn('month', F.month('created_utc'))
df = df.withColumn('year', F.year('created_utc'))

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 25, Finished, Available)

Converting media column to boolean:

In [21]:
# convert missing values to False
df = df.withColumn('has_media', F.col('media').isNotNull())

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 26, Finished, Available)

In [22]:
# drop unnecessary columns
df = df.drop(*["media", "created_utc", "disable_comments", "distinguished"])

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 27, Finished, Available)

In [23]:
df.printSchema()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 28, Finished, Available)

root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- num_comments: long (nullable = true)
 |-- score: long (nullable = true)
 |-- over_18: boolean (nullable = true)
 |-- pinned: boolean (nullable = true)
 |-- locked: boolean (nullable = true)
 |-- domain: string (nullable = true)
 |-- hidden: boolean (nullable = true)
 |-- hide_score: boolean (nullable = true)
 |-- post_length: integer (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- day_of_week_str: string (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- has_media: boolean (nullable = false)



## Read CSVs exported from the Spark Job

In [11]:
# Define the path to your CSV file
csv_file_path_1 = f"{datastore}csv/subreddit_count_eda.csv"
csv_file_path_2 = f"{datastore}csv/num_missing_val_eda.csv"
csv_file_path_3 = f"{datastore}csv/datetime_counts_eda.csv"

# Read the CSV file using PySpark
df_subreddit_count = spark.read.csv(csv_file_path_1, header=True, inferSchema=True)
df_num_missing = spark.read.csv(csv_file_path_2, header=True, inferSchema=True)
df_datetime = spark.read.csv(csv_file_path_3, header=True, inferSchema=True)

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 16, Finished, Available)

In [12]:
df_subreddit_count_pd = df_subreddit_count.toPandas()
df_num_missing_pd = df_num_missing.toPandas()
df_datetime_pd = df_datetime.toPandas()

StatementMeta(9d2667d4-d95f-4c64-adf7-bfab734cf5c0, 47, 17, Finished, Available)